Use Template Matching

Extract individual characters/words from the original text.
Use OpenCV's matchTemplate to locate exact positions in the augmented images.
Refine Masks

Generate precise segmentation masks by marking matched regions.
Apply morphological operations to clean up noise.
Validate Results

Use connected component analysis for accurate segmentation.
Utilize histogram profiles (horizontal & vertical) to refine character boundaries.


In [3]:
import cv2
import numpy as np
import os
import json

In [4]:


# Paths
text_images_folder = "../artifacts/rendered_images"  # Folder containing original text images
coords_folder = "../artifacts/coordinates"            # Folder with character bounding boxes (from OCR)
template_folder = "../artifacts/templates"            # Output folder for character templates
refined_masks_folder = "../artifacts/refined_masks"   # Output folder for refined masks

# Ensure refined masks folder exists
os.makedirs(refined_masks_folder, exist_ok=True)

def extract_characters(image_path, coords_path, template_folder):
    """Extract individual character templates from text images using saved coordinates."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if not os.path.exists(coords_path):
        print(f"Skipping {image_path} - No coordinates found.")
        return
    
    with open(coords_path, "r") as f:
        coordinates = json.load(f)

    for idx, box in enumerate(coordinates):
        x, y, w, h = box["x"], box["y"], box["width"], box["height"]
        char_crop = image[y:y+h, x:x+w]  # Extract character

        if char_crop.size == 0:
            continue  # Skip empty regions

        # Save character as an individual template
        template_path = os.path.join(template_folder, f"{os.path.basename(image_path).replace('.png', '')}_char{idx}.png")
        cv2.imwrite(template_path, char_crop)

    print(f"Templates extracted from {image_path} and saved in {template_folder}")

def refine_masks_with_template_matching(image_path, template_folder, refined_masks_folder):
    """Refine the mask of the text image using template matching."""
    original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    refined_mask = np.zeros_like(original_image)  # Initialize a blank mask

    # Iterate through all templates in the template folder
    for template_file in os.listdir(template_folder):
        if template_file.endswith(".png"):
            template_path = os.path.join(template_folder, template_file)
            template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
            w, h = template.shape[::-1]

            # Match the template
            result = cv2.matchTemplate(original_image, template, cv2.TM_CCOEFF_NORMED)
            threshold = 0.8  # Set a threshold for matching
            loc = np.where(result >= threshold)

            # Mark matched regions in the refined mask
            for pt in zip(*loc[::-1]):  # Switch columns and rows
                cv2.rectangle(refined_mask, pt, (pt[0] + w, pt[1] + h), 255, -1)

    # Save the refined mask
    refined_mask_path = os.path.join(refined_masks_folder, os.path.basename(image_path))
    cv2.imwrite(refined_mask_path, refined_mask)
    print(f"Refined mask saved at {refined_mask_path}")

# Process all images for character extraction and refine masks
for image_file in os.listdir(text_images_folder):
    if image_file.endswith(".png"):
        image_path = os.path.join(text_images_folder, image_file)
        coords_path = os.path.join(coords_folder, image_file.replace(".png", ".json"))
        extract_characters(image_path, coords_path, template_folder)
        
        # Refine masks using the extracted templates
        refine_masks_with_template_matching(image_path, template_folder, refined_masks_folder)

print("Character template extraction and mask refinement complete.")


Templates extracted from ../artifacts/rendered_images/4400.png and saved in ../artifacts/templates
Refined mask saved at ../artifacts/refined_masks/4400.png
Templates extracted from ../artifacts/rendered_images/8789.png and saved in ../artifacts/templates
Refined mask saved at ../artifacts/refined_masks/8789.png
Templates extracted from ../artifacts/rendered_images/6932.png and saved in ../artifacts/templates
Refined mask saved at ../artifacts/refined_masks/6932.png
Templates extracted from ../artifacts/rendered_images/3058.png and saved in ../artifacts/templates
Refined mask saved at ../artifacts/refined_masks/3058.png
Templates extracted from ../artifacts/rendered_images/3299.png and saved in ../artifacts/templates
Refined mask saved at ../artifacts/refined_masks/3299.png
Templates extracted from ../artifacts/rendered_images/1372.png and saved in ../artifacts/templates
Refined mask saved at ../artifacts/refined_masks/1372.png
Templates extracted from ../artifacts/rendered_images/7299